In [2]:
# -------------------------------
# 0. Install dependencies
# -------------------------------
!pip install torch scikit-learn pandas

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# -------------------------------
# 1. Load NSL-KDD from GitHub
# -------------------------------
url_train = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
url_test = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

col_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
    "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login",
    "is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"
]

train_df = pd.read_csv(url_train, names=col_names)
test_df = pd.read_csv(url_test, names=col_names)

# -------------------------------
# 2. Preprocess
# -------------------------------
# Binary labels: normal=0, attack=1
train_df['label'] = train_df['label'].apply(lambda x: 0 if x=='normal' else 1)
test_df['label'] = test_df['label'].apply(lambda x: 0 if x=='normal' else 1)

# Small subset for demo
train_df = train_df.sample(n=2000, random_state=42)
test_df = test_df.sample(n=500, random_state=42)

# Categorical and numerical features
cat_features = ['protocol_type', 'service', 'flag']
num_features = [c for c in col_names if c not in cat_features + ['label','difficulty']]

# -------------------------------
# Handle unseen categories by fitting on train + test
# -------------------------------
cat_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    le.fit(pd.concat([train_df[col], test_df[col]]))
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    cat_encoders[col] = le

# Normalize numerical features
scaler = MinMaxScaler()
train_df[num_features] = scaler.fit_transform(train_df[num_features])
test_df[num_features] = scaler.transform(test_df[num_features])

# Split into arrays
X_cat_train = torch.tensor(train_df[cat_features].values, dtype=torch.long)
X_cat_test = torch.tensor(test_df[cat_features].values, dtype=torch.long)
X_num_train = torch.tensor(train_df[num_features].values, dtype=torch.float32)
X_num_test = torch.tensor(test_df[num_features].values, dtype=torch.float32)
y_train = torch.tensor(train_df['label'].values, dtype=torch.long)
y_test = torch.tensor(test_df['label'].values, dtype=torch.long)

# -------------------------------
# 3. Define Transformer-based model
# -------------------------------
class IDS_Transformer(nn.Module):
    def __init__(self, num_features, cat_dims, embed_dim=16, num_heads=2, num_layers=1, num_classes=2):
        super().__init__()
        # Categorical embeddings
        self.cat_embeddings = nn.ModuleList([nn.Embedding(cat_size, embed_dim) for cat_size in cat_dims])
        # Numerical features projection
        self.num_proj = nn.Linear(num_features, embed_dim)
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=64, dropout=0.1)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(embed_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, num_classes)
        )

    def forward(self, X_num, X_cat):
        # Embed categorical features
        cat_embeds = [emb(X_cat[:, i]) for i, emb in enumerate(self.cat_embeddings)]
        cat_embeds = torch.stack(cat_embeds, dim=1)  # [batch, num_cat, embed_dim]
        # Project numerical features
        num_embed = self.num_proj(X_num).unsqueeze(1)  # [batch, 1, embed_dim]
        # Combine tokens
        tokens = torch.cat([num_embed, cat_embeds], dim=1)  # [batch, tokens, embed_dim]
        # Transformer expects [seq_len, batch, embed_dim]
        tokens = tokens.permute(1,0,2)
        out = self.transformer(tokens)
        # Pooling
        pooled = out.mean(dim=0)
        return self.fc(pooled)

# Instantiate model
cat_dims = [len(cat_encoders[col].classes_) for col in cat_features]
model = IDS_Transformer(num_features=X_num_train.shape[1], cat_dims=cat_dims)

# -------------------------------
# 4. Training setup
# -------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 5  # small demo

# -------------------------------
# 5. Training loop
# -------------------------------
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_num_train, X_cat_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Evaluate
    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_num_test, X_cat_test), dim=1)
        acc = (preds == y_test).float().mean()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Test Accuracy: {acc.item():.4f}")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/5, Loss: 0.6578, Test Accuracy: 0.7880
Epoch 2/5, Loss: 0.6429, Test Accuracy: 0.7760
Epoch 3/5, Loss: 0.6305, Test Accuracy: 0.7480
Epoch 4/5, Loss: 0.6169, Test Accuracy: 0.7220
Epoch 5/5, Loss: 0.6052, Test Accuracy: 0.7120
